In [1]:
# ==============================================================================
# CELLA 0: SETUP TOTALE (MINIMAL & STABILE)
# ==============================================================================
import sys
import os
from IPython.display import clear_output

# 1. BLOCCO MODULI PROBLEMATICI
sys.modules["vllm"] = None
sys.modules["vllm.sampling_params"] = None

print("⏳ Setup Ambiente in corso... (Attendere, output nascosto)")

# 2. INSTALLAZIONE & AGGIORNAMENTO SILENZIOSO
# Scarica l'ultima versione di Unsloth da Git e aggiorna automaticamente 
# PyTorch e Transformers alle versioni più recenti e compatibili.
!pip install --upgrade --no-cache-dir --quiet \
    "torch" "torchvision" "torchaudio" \
    "transformers" "trl" "peft" "accelerate" "bitsandbytes" \
    "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" \
    "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git" \
    "pillow" "scikit-learn" "pandas"

# 3. VERIFICA E PULIZIA
clear_output()

import torch
import unsloth
import transformers
from PIL import Image

print(f"✅ Ambiente Pronto e Pulito.")
print(f"   • GPU: {torch.cuda.get_device_name(0)}")
print(f"   • PyTorch: {torch.__version__}")
print(f"   • Unsloth: {unsloth.__version__}")
print(f"   • Transformers: {transformers.__version__}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/opt/conda/lib/python3.11/site-packages/pandas/core/computation/expressions.py:22: UserWarning: Pandas requires version '2.10.2' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Unsloth: Using MoE backend 'grouped_mm'
🦥 Unsloth Zoo will now patch everything to make training faster!
✅ Ambiente Pronto e Pulito.
   • GPU: Tesla V100S-PCIE-32GB
   • PyTorch: 2.10.0+cu128
   • Unsloth: 2026.2.1
   • Transformers: 4.57.6


In [2]:
import torch
import os
import gc
import json
import shutil
import time
import random 
import numpy as np 
import unsloth
from datetime import datetime
from datasets import load_from_disk
from trl import SFTTrainer, SFTConfig
from unsloth import FastVisionModel, UnslothVisionDataCollator, is_bfloat16_supported
from transformers import TrainerCallback, set_seed 

# ==============================================================================
# 1. CONFIGURAZIONE GLOBALE (Fissa per tutte le run)
# ==============================================================================
SEEDS = [101, 285, 3692, 92]  # <--- LISTA DI SEED DA TESTARE
NUM_EPOCHS = 5                # <--- NUMERO DI EPOCHE PER OGNI RUN (con SEED X)
MODEL_ID = "unsloth/Qwen2.5-VL-7B-Instruct-bnb-4bit"
MODEL_SHORTNAME = "Qwen2.5-VL-M2-Classification"
DATASET_PATH = os.path.join("DATASET_ITA", "PROCESSED_DATA", "HF_DATASETS", "M2_classification") # <--- CAMBIATO DATASET

# SYSTEM PROMPT (SPECIFICO PER M2 - CLASSIFICAZIONE 4 CLASSI)
SYSTEM_INSTRUCTION_M2 = """Sei un classificatore esperto specializzato nella tipologia di contenuti offensivi online.
Il contenuto che analizzerai (testo del commento e frame del video) è GIA' stato identificato come offensivo.
Il tuo compito è classificare ESATTAMENTE il tipo di offesa in una delle seguenti 4 categorie:

1. FLAMING: Insulti diretti, linguaggio ostile, aggressività verbale, minacce, uso di parolacce contro una persona.
2. DENIGRATION: Attacchi alla reputazione, ridicolizzazione, svalutazione, diffamazione o umiliazione pubblica.
3. SEXUAL: Molestie sessuali, commenti lascivi, oggettivazione sessuale, riferimenti espliciti non consensuali.
4. RACISM: Discriminazione, stereotipi o insulti basati su razza, etnia, nazionalità, religione o colore della pelle.

Analizza CONGIUNTAMENTE il testo del commento e i frame del video.
Scegli la categoria che meglio descrive l'offesa predominante.
Se più categorie sono presenti, scegli quella DOMINANTE.

Formato di Output OBBLIGATORIO:
Rispondi SOLAMENTE con il numero della classe (1, 2, 3 o 4).
Non aggiungere spiegazioni, punteggiatura o testo extra."""

# Callback per monitoraggio
class RealTimePrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"📝 Step: {state.global_step:4d} | Epoch: {logs['epoch']:.2f} | Loss: {logs['loss']:.4f}")

# ==============================================================================
# 2. CARICAMENTO E FORMATTAZIONE DATASET (Una volta sola per efficienza)
# ==============================================================================
print("📂 Caricamento Dataset HF M2 (Eseguito una volta sola)...")
dataset_raw = load_from_disk(DATASET_PATH)

def has_valid_images(sample):
    user_msg = sample["messages"][0]
    for item in user_msg["content"]:
        if item["type"] == "image":
            raw_path = item["image"]
            clean_path = raw_path.replace("file://", "")
            check_path = "/" + clean_path.lstrip("/") if clean_path else ""
            if not os.path.exists(check_path):
                return False
    return True

# Filtriamo eventuali immagini rotte
train_valid = dataset_raw["train"].filter(has_valid_images, desc="Filter Valid Imgs")
val_valid = dataset_raw["val"].filter(has_valid_images, desc="Filter Valid Imgs")

def format_multimodal_sample_m2(sample):
    raw_user_msg = sample["messages"][0]
    raw_assistant_msg = sample["messages"][1] 
    user_content = []
    
    for item in raw_user_msg["content"]:
        if item["type"] == "image":
            raw_path = item["image"]
            clean_path = raw_path.replace("file://", "")
            clean_path = "/" + clean_path.lstrip("/") 
            final_path = f"file://{clean_path}"
            user_content.append({"type": "image", "image": final_path})
        elif item["type"] == "text":
            text_clean = item["text"].replace("Commento:", "").strip().strip('"').strip("'")
            text_final = f"Commento: \"{text_clean}\""
            user_content.append({"type": "text", "text": text_final})
            
    # Per M2 la label sarà "1", "2", "3" o "4"
    label_text = raw_assistant_msg["content"][0]["text"]

    new_messages = [
        # QUI USIAMO IL PROMPT M2
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCTION_M2}]},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": [{"type": "text", "text": label_text}]}
    ]
    return {"messages": new_messages}

print("🔄 Formattazione Dataset M2...")
train_dataset = train_valid.map(format_multimodal_sample_m2, batched=False, desc="Formatting Train M2")
val_dataset = val_valid.map(format_multimodal_sample_m2, batched=False, desc="Formatting Val M2")
print(f"✅ Dataset M2 Caricato e Formattato. Train: {len(train_dataset)} | Val: {len(val_dataset)}")


# ==============================================================================
# 3. MEGA-LOOP DI TRAINING (TUTTI I SEEDS)
# ==============================================================================
print(f"\n🚀 AVVIO SESSIONE DI TRAINING M2 SU {len(SEEDS)} SEED: {SEEDS}")

for seed_idx, TRAINING_SEED in enumerate(SEEDS):
    print("\n" + "#"*60)
    print(f"🎬 RUN {seed_idx + 1}/{len(SEEDS)} | SEED CORRENTE: {TRAINING_SEED}")
    print("#"*60)

    # --- 🔒 FIX DETERMINISMO GLOBALE ---
    print(f"🔒 Fissaggio Seed Globali a {TRAINING_SEED}...")
    random.seed(TRAINING_SEED)
    np.random.seed(TRAINING_SEED)
    torch.manual_seed(TRAINING_SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(TRAINING_SEED)
    set_seed(TRAINING_SEED)
    # -------------------------------------------------------

    # Definizione Output Directory Dinamica (M2)
    OUTPUT_DIR = f"outputs/{MODEL_SHORTNAME}_Seed_{TRAINING_SEED}"
    print(f"📂 Cartella Output Run: {OUTPUT_DIR}")

    # --- A. CARICAMENTO MODELLO ---
    print(f"⏳ Inizializzazione Modello M2 (Seed {TRAINING_SEED})...")
    model, tokenizer = FastVisionModel.from_pretrained(
        model_name = MODEL_ID,
        load_in_4bit = True,
        use_gradient_checkpointing = "unsloth",
    )

    model = FastVisionModel.get_peft_model(
        model,
        r = 16,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha = 16,
        lora_dropout = 0,
        bias = "none",
        random_state = TRAINING_SEED, 
        use_rslora = False,
        loftq_config = None,
    )
    FastVisionModel.for_training(model)

    # --- B. CONFIGURAZIONE TRAINER ---
    training_args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        num_train_epochs = NUM_EPOCHS,
        learning_rate = 5e-5,
        lr_scheduler_type = "cosine",
        warmup_ratio = 0.1,
        weight_decay = 0.01,
        optim = "adamw_8bit",
        max_grad_norm = 0.3,
        
        # Salvataggio
        eval_strategy = "epoch",
        save_strategy = "epoch",
        save_total_limit = None,
        load_best_model_at_end = False,
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        
        # Hardware & Path
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        gradient_checkpointing = True,
        logging_steps = 10,
        output_dir = OUTPUT_DIR, 
        report_to = "none",
        
        # Unsloth
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        seed = TRAINING_SEED, 
    )

    trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        data_collator = UnslothVisionDataCollator(model, tokenizer),
        train_dataset = train_dataset,
        eval_dataset = val_dataset,
        args = training_args,
        callbacks = [RealTimePrinterCallback()],
    )

    # --- C. ESECUZIONE TRAINING ---
    print(f"🔥 Avvio Training M2 Seed {TRAINING_SEED}...")
    torch.cuda.empty_cache()
    
    start_time = time.time()
    trainer_stats = trainer.train()
    end_time = time.time()
    
    total_duration = (end_time - start_time) / 60
    final_train_loss = trainer_stats.training_loss
    global_steps_done = trainer_stats.global_step

    print(f"✅ Training M2 Finito. Durata: {total_duration:.2f} min | Loss: {final_train_loss:.4f}")

    # --- D. SALVATAGGIO ---
    ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_latest")
    REPORT_FILENAME = f"training_report_Seed_{TRAINING_SEED}.json"
    REPORT_PATH = os.path.join(OUTPUT_DIR, REPORT_FILENAME)
    ZIP_FILENAME = f"{MODEL_SHORTNAME}_Seed_{TRAINING_SEED}_FULL_CHECKPOINTS"
    
    # Cartella Padre per lo ZIP
    PARENT_DIR = os.path.dirname(OUTPUT_DIR)
    ZIP_FULL_PATH = os.path.join(PARENT_DIR, ZIP_FILENAME)

    os.makedirs(ADAPTER_PATH, exist_ok=True)
    
    print(f"💾 Salvataggio Artifacts M2...")
    model.save_pretrained(ADAPTER_PATH)
    tokenizer.save_pretrained(ADAPTER_PATH)

    # Report JSON
    peft_config_data = "N/A"
    try:
        raw_config = getattr(model, "peft_config", None)
        if isinstance(raw_config, dict) and raw_config.get("default"):
            peft_config_data = str(raw_config["default"])
    except: pass

    full_report = {
        "1_META_INFO": {
            "timestamp_end": datetime.now().isoformat(),
            "model_shortname": MODEL_SHORTNAME,
            "seed": TRAINING_SEED,
            "task": "M2 Classification (1-4) - Training Loop" # <--- TASK CORRETTO
        },
        "4_TRAINING_PERFORMANCE": {
            "total_duration_minutes": total_duration,
            "final_training_loss": final_train_loss,
            "global_steps": global_steps_done,
            "epochs": training_args.num_train_epochs
        },
        "5_LORA_PARAMS": peft_config_data,
        "6_SYSTEM_PROMPT": SYSTEM_INSTRUCTION_M2, # <--- SALVIAMO PROMPT M2
        "7_ARTIFACTS": {
            "checkpoints_location": "Inside ZIP archive",
            "zip_path": f"{ZIP_FULL_PATH}.zip"
        },
        "8_FULL_LOG_HISTORY": getattr(trainer.state, "log_history", [])
    }

    with open(REPORT_PATH, "w", encoding="utf-8") as f:
        json.dump(full_report, f, indent=4, ensure_ascii=False)

    print(f"📦 Compressione ZIP in corso (attendere)...")
    shutil.make_archive(
        base_name=ZIP_FULL_PATH, 
        format='zip', 
        root_dir=OUTPUT_DIR
    )
    print(f"   -> ZIP creato: {ZIP_FULL_PATH}.zip")

    # --- E. PULIZIA MEMORIA ---
    print(f"🧹 Pulizia VRAM per il prossimo seed...")
    try:
        del model
        del trainer
        del tokenizer
    except: pass
    
    gc.collect()
    torch.cuda.empty_cache()
    print("✨ Ambiente pulito.\n")

print("\n🎉 TUTTE LE RUN PER M2 SONO COMPLETATE CON SUCCESSO!")

📂 Caricamento Dataset HF M2 (Eseguito una volta sola)...
🔄 Formattazione Dataset M2...
✅ Dataset M2 Caricato e Formattato. Train: 967 | Val: 168

🚀 AVVIO SESSIONE DI TRAINING M2 SU 4 SEED: [101, 285, 3692, 92]

############################################################
🎬 RUN 1/4 | SEED CORRENTE: 101
############################################################
🔒 Fissaggio Seed Globali a 101...
📂 Cartella Output Run: outputs/Qwen2.5-VL-M2-Classification_Seed_101
⏳ Inizializzazione Modello M2 (Seed 101)...
==((====))==  Unsloth 2026.2.1: Fast Qwen2_5_Vl patching. Transformers: 4.57.6. vLLM: 0.6.3.
   \\   /|    Tesla V100S-PCIE-32GB. Num GPUs = 1. Max memory: 31.739 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.0. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: 

The model is already on multiple devices. Skipping the move to device specified in `args`.


🔥 Avvio Training M2 Seed 101...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 967 | Num Epochs = 5 | Total steps = 305
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 47,589,376 of 8,339,756,032 (0.57% trained)


Unsloth: Will smartly offload gradients to save VRAM!


...

🔥 Avvio Training M2 Seed 92...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 967 | Num Epochs = 5 | Total steps = 305
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 47,589,376 of 8,339,756,032 (0.57% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss
1,0.220400,0.151645
2,0.140800,0.127651
3,0.137700,0.125958
4,0.131800,0.125897
5,0.142800,0.125715


📝 Step:   10 | Epoch: 0.17 | Loss: 4.2155
📝 Step:   20 | Epoch: 0.33 | Loss: 2.9547
📝 Step:   30 | Epoch: 0.50 | Loss: 2.0164
📝 Step:   40 | Epoch: 0.66 | Loss: 1.3349
📝 Step:   50 | Epoch: 0.83 | Loss: 0.5830
📝 Step:   60 | Epoch: 0.99 | Loss: 0.2204
📝 Step:   70 | Epoch: 1.15 | Loss: 0.1674
📝 Step:   80 | Epoch: 1.31 | Loss: 0.1488
📝 Step:   90 | Epoch: 1.48 | Loss: 0.1426
📝 Step:  100 | Epoch: 1.64 | Loss: 0.1414
📝 Step:  110 | Epoch: 1.81 | Loss: 0.1515
📝 Step:  120 | Epoch: 1.98 | Loss: 0.1408
📝 Step:  130 | Epoch: 2.13 | Loss: 0.1408
📝 Step:  140 | Epoch: 2.30 | Loss: 0.1385
📝 Step:  150 | Epoch: 2.46 | Loss: 0.1416
📝 Step:  160 | Epoch: 2.63 | Loss: 0.1200
📝 Step:  170 | Epoch: 2.79 | Loss: 0.1371
📝 Step:  180 | Epoch: 2.96 | Loss: 0.1377
📝 Step:  190 | Epoch: 3.12 | Loss: 0.1499
📝 Step:  200 | Epoch: 3.28 | Loss: 0.1238
📝 Step:  210 | Epoch: 3.45 | Loss: 0.1295
📝 Step:  220 | Epoch: 3.61 | Loss: 0.1188
📝 Step:  230 | Epoch: 3.78 | Loss: 0.1402
📝 Step:  240 | Epoch: 3.94 | Loss: